In [1]:
import os, sys, shutil
# sys.path.append(os.path.join(os.getcwd(),"inputs"))
# sys.path.append(os.path.join(os.getcwd(),r"../../../scripts/summarize/standard"))
# sys.path.append(os.path.join(os.getcwd(),r'../../..'))
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
from shapely import wkt
# from input_configuration import base_year
import toml
config = toml.load(os.path.join(os.getcwd(),'../../../../configuration/input_configuration.toml'))
summary_config = toml.load(os.path.join(os.getcwd(),'../../../../configuration/summary_configuration.toml'))

# from emme_configuration import *
# from standard_summary_configuration import *
pd.options.mode.chained_assignment = None
import os, sys, time, shutil
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None
import h5py

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
output_path = os.path.join(summary_config['sc_run_path'], summary_config["output_folder"])
survey_path = os.path.join(summary_config['sc_run_path'], summary_config["survey_folder"])

In [3]:
def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Congested network components by time of day
    df.columns

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    
    return df

In [4]:
pd.options.display.float_format = '{:0,.0f}'.format

## Regional Emissions
Only including light, medium, and heavy vehicles; bus vehicles are excluded.

In [5]:
emissions_summary = pd.read_csv(os.path.join(output_path,'emissions','emissions_summary.csv'))

df = emissions_summary.copy()
df = df[df['veh_type'].isin(['light','medium','heavy'])]
df = df.groupby('pollutant_name').sum()
df.rename(columns={'start_tons': 'Start', 'intrazonal_tons': 'Intrazonal', 'interzonal_tons': 'Interzonal',
                  'total_daily_tons': 'Total Daily'}, inplace=True)
df = df[df.index.isin(['CO','NOx','PM25 Total','PM10 Total','CO2 Equivalent','VOCs'])]

# FIXME line below is failing at 3.11. I dont see a need for it since there are no decimals in the output.
#df = df.applymap(lambda x: x if x > 100 else str(round(x,1)))
df

,pollutantID,veh_type,Start,Intrazonal,Interzonal,Total Daily
pollutant_name,,,,,,
CO,222,heavylightmedium,191,3,281,474
CO2 Equivalent,989898,heavylightmedium,"2,726",271,"38,529","41,526"
NOx,333,heavylightmedium,11,0,67,78
PM10 Total,PM10PM10PM10,heavylightmedium,0,0,5,6
PM25 Total,PM25PM25PM25,heavylightmedium,0,0,2,2
VOCs,878787,heavylightmedium,11,0,8,19


## Emissions by Vehicle Type

In [6]:
# Calculate emissions and VMT by vehicle type and save results
# Note that Total VMT will not match regional totals because we are not included buses in the emissions summaries

df = emissions_summary.copy()
df = df.groupby(['pollutant_name','veh_type']).sum()
df.rename(columns={'start_tons': 'Start', 'intrazonal_tons': 'Intrazonal', 'interzonal_tons': 'Interzonal',
                  'total_daily_tons': 'Total Daily'}, inplace=True)
df = df.reset_index()
df = df[df.pollutant_name.isin(['CO','NOx','PM25 Total','PM10 Total','CO2 Equivalent','VOCs'])]
# df
df_network = load_network_summary(os.path.join(output_path,'network',r'network_results.csv'))

df_network['light_vmt'] = df_network[['@hov2_inc1','@hov2_inc2', '@hov2_inc3', '@hov3_inc1', '@hov3_inc2', '@hov3_inc3',
          '@sov_inc1', '@sov_inc2', '@sov_inc3', '@tnc_inc1', '@tnc_inc2','@tnc_inc3']].sum(axis=1)
df_network['light_vmt'] = df_network['light_vmt']*df_network['length']

df_network['medium_vmt'] = df_network['@mveh']*df_network['length']

df_network['heavy_vmt'] = df_network['@hveh']*df_network['length']

_df = pd.DataFrame([df_network['light_vmt'].sum(),
              df_network['medium_vmt'].sum(),
              df_network['heavy_vmt'].sum()])
_df.index = ['light','medium','heavy']
_df.columns=['VMT']
_df['veh_type'] = _df.index

df = df.merge(_df, on='veh_type', how='left')

df.to_csv(os.path.join(output_path,'emissions',r'emissions_by_vehtype_summary.csv'))

df.rename(columns={'pollutant_name': 'Pollutant', 'veh_type': 'Vehicle Type',
                   'Total Daily': 'Total Daily (Tons)'}, inplace=True)
col_list = ['Start','Intrazonal','Start','Intrazonal','Interzonal','Total Daily (Tons)', 'VMT']

# FIXME line below is failing at 3.11. I dont see a need for it since there are no decimals in the output. 
#df[col_list] = df[col_list].applymap(lambda x: x if x > 100 else str(round(x,1)))
df

,Pollutant,Vehicle Type,pollutantID,Start,Intrazonal,Interzonal,Total Daily (Tons),VMT
0,CO,heavy,2,0,0,8,8,"2,809,617"
1,CO,light,2,176,3,258,436,"76,221,675"
2,CO,medium,2,15,0,15,30,"3,450,243"
3,CO,transit,2,0,0,1,2,NaN
4,CO2 Equivalent,heavy,98,3,0,"5,338","5,341","2,809,617"
5,CO2 Equivalent,light,98,"2,565",271,"30,937","33,774","76,221,675"
6,CO2 Equivalent,medium,98,157,0,"2,253","2,411","3,450,243"
7,CO2 Equivalent,transit,98,4,0,344,347,NaN
8,NOx,heavy,3,0,0,28,28,"2,809,617"
9,NOx,light,3,10,0,34,44,"76,221,675"
